In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4ohmgxzu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4ohmgxzu
     |████████████████████████████████| 129 kB 34.5 MB/s 
     |████████████████████████████████| 344 kB 65.9 MB/s 
     |████████████████████████████████| 54.7 MB 37 kB/s 
     |████████████████████████████████| 4.5 MB 46.2 MB/s 
     |████████████████████████████████| 1.2 MB 27.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x396d2000 @  0x7fc6c023d615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 

In [2]:
!git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 428 (delta 129), reused 108 (delta 106), pack-reused 275
Receiving objects: 100% (428/428), 219.68 KiB | 957.00 KiB/s, done.
Resolving deltas: 100% (220/220), done.


In [3]:
!pip install -r /content/KoBERT/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [8]:
data_train = pd.read_csv('/content/data_train.tsv',sep='\t')
data_test = pd.read_csv('/content/data_test.tsv',sep='\t')

In [9]:
import re
def replace_special(txt):
    return re.sub('[^가-힣]', "", txt)

In [10]:
data_train['text'] = data_train['text'].apply(replace_special)

In [11]:
data_test['text'] = data_test['text'].apply(replace_special)

In [12]:
data_train.drop_duplicates(inplace=True)

In [13]:
data_test.drop_duplicates(inplace=True)

In [14]:
data_train['text'] = data_train['text'].str.replace('^ +',"")
data_train['text'].replace('',np.nan,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [15]:
data_test['text'] = data_test['text'].str.replace('^ +',"")
data_test['text'].replace('',np.nan,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [16]:
data_train.dropna(inplace=True)

In [17]:
data_test.dropna(inplace=True)

In [18]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
data_train.to_csv('train.tsv',index=False,sep='\t')
data_test.to_csv('test.tsv',index=False,sep='\t')

In [20]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15960 entries, 0 to 20088
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15960 non-null  object
 1   label   15960 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 374.1+ KB


In [ ]:
data_train['len'] = data_train['text'].apply(lambda x : len(x))

In [ ]:
data_train['len'].max()

82

In [21]:
# field_indices : [학습시킬 데이터의 index, 데이터 레이블의 index]
# num_discard_samples : 데이터 상단에서 제외할 row의 개수 (default = 0)

dataset_train = nlp.data.TSVDataset("/content/train.tsv", field_indices=[0,1], num_discard_samples=1,allow_missing=True)
dataset_test = nlp.data.TSVDataset("/content/test.tsv", field_indices=[1,2], num_discard_samples=1,allow_missing=True)

In [74]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([data[sent_idx]]) for data in dataset]
        self.labels = [np.int32(data[label_idx]) for data in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [23]:
## Setting parameters
max_len = 84 #문자열 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 10
learning_rate =  5e-5


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
## CPU
#device = torch.device("cpu")

## GPU
# device = torch.device("cuda:0")

In [75]:
'''
dataset = TSV 파일 (TSV파일은 id, sentance, label 순으로 저장되어 있음. 불러올땐, sentance, label만 불러옴)
sent_idx = dataset에서 sentance의 위치
label_idx = dataset에서 label 위치
bert_tokenizer = 토크나이저 객체
max_len = 문자열 최대 길이
pad = max_len으로부터 남는 문자열의 길이에 대한 패딩 유무
pair = Whether to transform sentences or sentence pairs.

https://nlp.gluon.ai/_modules/gluonnlp/data/transforms.html#BERTSentenceTransform

'''


data_train = BERTDataset(dataset=dataset_train, sent_idx=0, label_idx=1, 
                         bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)

data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [66]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=1)

In [60]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)   # <-- return_dict=False 추가했습니다
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [61]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [62]:
# Prepare optimizer and schedule (linear warmup and decay)

'''
model.named_parameters() 중에 'bias', 'LayerNorm.weight' 라는 이름을 가진 파라미터의 유무를 나눠서 리스트로 저장
'''


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [63]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [67]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [68]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [69]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id,(token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
       
        valid_length = valid_length
        
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        test_acc += calc_accuracy(out, label)
        
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/250 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6822537779808044 train acc 0.5625
epoch 1 batch id 11 loss 0.6598192453384399 train acc 0.5795454545454546
epoch 1 batch id 21 loss 0.6755547523498535 train acc 0.6004464285714286
epoch 1 batch id 31 loss 0.6077510714530945 train acc 0.6159274193548387
epoch 1 batch id 41 loss 0.6172518730163574 train acc 0.6269054878048781
epoch 1 batch id 51 loss 0.6091143488883972 train acc 0.6387867647058824
epoch 1 batch id 61 loss 0.5626578330993652 train acc 0.6454918032786885
epoch 1 batch id 71 loss 0.5249131321907043 train acc 0.6536091549295775
epoch 1 batch id 81 loss 0.45890673995018005 train acc 0.6653163580246914
epoch 1 batch id 91 loss 0.4913855493068695 train acc 0.6809752747252747
epoch 1 batch id 101 loss 0.42983630299568176 train acc 0.6918316831683168
epoch 1 batch id 111 loss 0.39756476879119873 train acc 0.7052364864864865
epoch 1 batch id 121 loss 0.4126249849796295 train acc 0.7166838842975206
epoch 1 batch id 131 loss 1.0235795974731445 train acc 0.7

  0%|          | 0/154 [00:00<?, ?it/s]

epoch 1 test acc 0.8799715909090909


  0%|          | 0/250 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9523133116883117


  0%|          | 0/250 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0073311892338097095 train acc 1.0
epoch 5 batch id 11 loss 0.03620758652687073 train acc 0.9928977272727273
epoch 5 batch id 21 loss 0.00493298564106226 train acc 0.9895833333333334
epoch 5 batch id 31 loss 0.0030589133966714144 train acc 0.9914314516129032
epoch 5 batch id 41 loss 0.005227584857493639 train acc 0.993140243902439
epoch 5 batch id 51 loss 0.10286446660757065 train acc 0.9932598039215687
epoch 5 batch id 61 loss 0.06874438375234604 train acc 0.9933401639344263
epoch 5 batch id 71 loss 0.12467576563358307 train acc 0.9911971830985915
epoch 5 batch id 81 loss 0.17208567261695862 train acc 0.9915123456790124
epoch 5 batch id 91 loss 0.04301397129893303 train acc 0.9896978021978022
epoch 5 batch id 101 loss 0.006140170153230429 train acc 0.989634900990099
epoch 5 batch id 111 loss 0.0026827137917280197 train acc 0.9901463963963963
epoch 5 batch id 121 loss 0.009783514775335789 train acc 0.9907024793388429
epoch 5 batch id 131 loss 0.0765927731990814

  0%|          | 0/154 [00:00<?, ?it/s]

epoch 5 test acc 0.9543425324675324
